In [12]:
import yfinance as yf
sp500 = yf.Ticker('AMZN')


sp500 = sp500.history(period='max')
# sp500.plot.line(y='Close', use_index=True)
del sp500['Dividends']
del sp500['Stock Splits']
sp500['Tomorrow'] = sp500['Close'].shift(-1)
sp500['Target'] = (sp500['Tomorrow'] > sp500['Close']).astype(int)
sp500 = sp500.loc['1990-01-01':].copy()

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

predictors = ['Close', 'Volume', 'Open', 'High', 'Low']
model.fit(train[predictors], train['Target'])

from sklearn.metrics import precision_score

preds = model.predict(test[predictors])

import pandas as pd
preds = pd.Series(preds, index=test.index)
precision_score(test['Target'], preds)

combined = pd.concat([test['Target'], preds], axis=1)

horizons = [2, 5, 60, 250, 1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = sp500.rolling(horizon).mean()

    ratio_column = f"Close_Ratio_{horizon}"
    sp500[ratio_column] = sp500['Close'] / rolling_averages['Close']

    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()['Target']

    new_predictors += [ratio_column, trend_column]

sp500 = sp500.dropna()

model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)



import pandas as pd
from ta.momentum import RSIIndicator

def predict(train, test, predictors, model, rsi_window=14):
    # Train the model
    model.fit(train[predictors], train['Target'])
    
    # Make predictions based on the model
    preds = model.predict_proba(test[predictors])[:, 1]

    # RSI Calculation
    test['RSI'] = RSIIndicator(test['Close'], window=rsi_window).rsi()

    # Set predictions to 1 if RSI is below 30
    preds[test['RSI'] < 30] = 1

    # Threshold adjustment (optional)
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0

    # Create a Series with predictions
    preds = pd.Series(preds, index=test.index, name='Predictions')

    # Combine actual Target values and predictions into a DataFrame
    combined = pd.concat([test[['Target', 'RSI']], preds], axis=1)

    return combined


# Example usage
# Assume 'Close' is the column representing the closing prices in your dataset
# and 'Target' is the column representing whether the S&P 500 is going up or not (1 for up, 0 for not).
# 'model' should be an instance of your machine learning model.

# result = predict(train_data, test_data, ['Close'], model)



def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)





In [13]:
predictions = backtest(sp500, model, new_predictors)
predictions['Predictions'].value_counts()

0.0    3025
1.0     196
Name: Predictions, dtype: int64

In [9]:
precision_score(predictions['Target'], predictions['Predictions'])

0.47368421052631576

In [1]:
import yfinance as yf

In [2]:
stock = yf.Ticker('AAPL')

In [4]:
stock.get_news()

[{'uuid': '6c953643-c7d8-3880-a062-ce626ce0385c',
  'title': 'PayPal Earnings Are Here. Investors Need Evidence It Can Compete With Big Tech.',
  'publisher': 'Barrons.com',
  'link': 'https://finance.yahoo.com/m/6c953643-c7d8-3880-a062-ce626ce0385c/paypal-earnings-are-here..html',
  'providerPublishTime': 1707285600,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/fzsYiUW0kDtWDhM0CRSuMA--~B/aD02NDA7dz0xMjgwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/Barrons.com/4e9aa326218f5ee63ca8371e5f7b7b24',
     'width': 1280,
     'height': 640,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/dmaB6uYUkbhcJMCW82mHQw--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/Barrons.com/4e9aa326218f5ee63ca8371e5f7b7b24',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['PYPL', 'AAPL']},
 {'uuid': 'ffcdb503-63f9-314f-8365-416479cb1e79',
  'title': 'TSMC’s Month